In [1]:
import sys
sys.path.append('../src/')
from catnip import *

sites = 6

print("\n> conversion checks:")
vector = random_vector(sites)
reconstructed_vector = mps_to_vector(vector_to_mps(vector))
print("vector-to-MPS-to-vector:             ", np.allclose(vector, reconstructed_vector))

mps = random_mps(sites, bond_limits = [hsd+1, 10])
reconstructed_mps = vector_to_mps(mps_to_vector(mps))
print("MPS-to-vector-to-MPS-to-vector:      ", np.allclose(mps_to_vector(mps), mps_to_vector(reconstructed_mps)) )

matrix = random_matrix(sites)
reconstructed_matrix = mpo_to_matrix(matrix_to_mpo(matrix))
print("matrix-to-MPO-to-matrix:             ", np.allclose(matrix, reconstructed_matrix))

mpo = random_mpo(sites, bond_limits = [hsd+1, 10])
reconstructed_mpo = matrix_to_mpo(mpo_to_matrix(mpo))
print("MPO-to-matrix-to-MPO-to-matrix:      ", np.allclose(mpo_to_matrix(mpo), mpo_to_matrix(reconstructed_mpo)) )


print("\n> product/contraction checks:")
vector = random_vector(sites)
matrix = random_matrix(sites)
result1 = matrix @ vector
result2 = mps_to_vector(mpo_mps_contraction(matrix_to_mpo(matrix), vector_to_mps(vector)))
print("vector-matrix product:               ",np.allclose(result1, result2))

mps = random_mps(sites, bond_limits = [hsd+1, 10])
mpo = random_mpo(sites, bond_limits = [hsd+1, 10])
result1 = mps_to_vector(mpo_mps_contraction(mpo, mps))
result2 = mpo_to_matrix(mpo) @ mps_to_vector(mps)
print("mps-mpo contraction:                 ",np.allclose(result1, result2))

matrixA = random_matrix(sites)
matrixB = random_matrix(sites)
result1 = matrixA @ matrixB
result2 = mpo_to_matrix(mpo_mpo_contraction(matrix_to_mpo(matrixA), matrix_to_mpo(matrixB)))
print("matrix-matrix product:               ",np.allclose(result1, result2))

mpoA = random_mpo(sites, bond_limits = [hsd+1, 10])
mpoB = random_mpo(sites, bond_limits = [hsd+1, 10])
result1 = mpo_to_matrix(mpoA) @ mpo_to_matrix(mpoB)
result2 = mpo_to_matrix(mpo_mpo_contraction(mpoA, mpoB))
print("mpo-mpo contraction:                 ",np.allclose(result1, result2))


> conversion checks:
vector-to-MPS-to-vector:              True
MPS-to-vector-to-MPS-to-vector:       True
matrix-to-MPO-to-matrix:              True
MPO-to-matrix-to-MPO-to-matrix:       True

> product/contraction checks:
vector-matrix product:                True
mps-mpo contraction:                  True
matrix-matrix product:                True
mpo-mpo contraction:                  True


In [2]:
sites = 6

print("\n> optimization checks:")
vector = random_vector(sites)
mps = vector_to_mps(vector)
print("vec-to-mps-optR-to-vec:              ", np.allclose(vector, mps_to_vector(optimize_mps_R(mps))))
print("vec-to-mps-optL-to-vec:              ", np.allclose(vector, mps_to_vector(optimize_mps_L(mps))))
optimized_mps, entanglement_entropy = mps_entanglement_entropy(mps)
print("vec-to-mps-optM-to-vec:              ", np.allclose(vector, mps_to_vector(optimized_mps)))


matrix = random_matrix(sites)
mpo = matrix_to_mpo(matrix)
print("matrix-to-mpo-optL-to-matrix:        ", np.allclose(matrix, mpo_to_matrix(optimize_mpo_L(mpo))))
print("matrix-to-mpo-optR-to-matrix:        ", np.allclose(matrix, mpo_to_matrix(optimize_mpo_R(mpo))))
optimized_mpo, entanglement_entropy = mpo_entanglement_entropy(mpo)
print("matrix-to-mpo-optM-to-matrix:        ", np.allclose(matrix, mpo_to_matrix(optimized_mpo)))



print("\n> final check:")

mps = random_mps(sites, bond_limits = [hsd+1, 10])
mpo1 = random_mpo(sites, bond_limits = [hsd+1, 10])
mpo2 = random_mpo(sites, bond_limits = [hsd+1, 10])

result1_mps, entanglement_entropy1 = mps_entanglement_entropy(
    mpo_mps_contraction(
        mpo_mpo_contraction(
            mpo1, mpo2
        ),
        mps
    )
)
result1 = mps_to_vector(result1_mps)

result2_mps, entanglement_entropy2 = mps_entanglement_entropy(
    optimize_mps_R(mpo_mps_contraction(
        mpo_mpo_contraction(
            optimize_mpo_L(mpo1), optimize_mpo_R(mpo2)
        ),
        optimize_mps_L(mps)
    ))
)
result2 = mps_to_vector(result2_mps)

print("everything together:                 ",np.allclose(result1, result2))
print("entanglement entropy equal:          ", np.allclose(entanglement_entropy1, entanglement_entropy2))





> optimization checks:
vec-to-mps-optR-to-vec:               True
vec-to-mps-optL-to-vec:               True
vec-to-mps-optM-to-vec:               True
matrix-to-mpo-optL-to-matrix:         True
matrix-to-mpo-optR-to-matrix:         True
matrix-to-mpo-optM-to-matrix:         True

> final check:
everything together:                  True
entanglement entropy equal:           True
